In [1]:
import numpy as np
import random
import torch
import argparse
import sys

from utils import seed_everything
from dataset import get_dataset
from community_detection import hierarchical_structure_generation
from execution import execute_NC, execute_LP
from preparation import LP_preparation, NC_preparation

import warnings
warnings.filterwarnings('ignore')

In [2]:
sys.argv = ['']  # execution on jupyter notebook
parser = argparse.ArgumentParser()
# general
parser.add_argument('--task', dest='task', default='LP', type=str,
                    help='LP; NC; Inductive')
parser.add_argument('--mode', dest='mode', default='baseline', type=str,
                    help='experiment mode. E.g., baseline or basemodel')
parser.add_argument('--model', dest='model', default='GCN', type=str,
                    help='model class name. E.g., GCN, PGNN, HCGNN...')
parser.add_argument('--dataset', dest='dataset', default='grid', type=str,
                    help='cora; grid; communities; ppi')
parser.add_argument('--gpu', dest='gpu', default=True, type=bool,
                    help='whether use gpu')
parser.add_argument('--SEED', dest='SEED', default=123, type=int)

# dataset
parser.add_argument('--ratio_sample_pos_link', dest='ratio_sample_pos_link', default=20, type=float)
parser.add_argument('--use_features', dest='use_features', default=True, type=bool,
                    help='whether use node features')
parser.add_argument('--community_detection_method', dest='community_detection_method', default='Louvain', type=str,
                    help='community detection method, default Louvain')
parser.add_argument('--threshold', dest='threshold', default=1, type=int,
                    help='the threshold for graph generation, default 1')

# model
parser.add_argument('--lr', dest='lr', default=1e-2, type=float)
parser.add_argument('--epoch_num', dest='epoch_num', default=201, type=int)
parser.add_argument('--epoch_log', dest='epoch_log', default=10, type=int)
parser.add_argument('--layer_num', dest='layer_num', default=2, type=int)
parser.add_argument('--relu', dest='relu', default=True, type=bool)
parser.add_argument('--dropout', dest='dropout', default=False, type=bool)
parser.add_argument('--drop_ratio', dest='drop_ratio', default=0.5, type=float)
parser.add_argument('--feature_pre', dest='feature_pre', default=True, type=bool)
parser.add_argument('--same_level_gnn', dest='same_level_gnn', default='GCN', type=str,
                    help='agg within level. E.g., MEAN GCN, SAGE, GAT, GIN, ...')
parser.add_argument('--down2up_gnn', dest='down2up_gnn', default='MEAN', type=str,
                    help='aggregation bottom-up. E.g., MEAN, GCN, SAGE, GAT, GIN, ...')
parser.add_argument('--up2down_gnn', dest='up2down_gnn', default='GAT', type=str,
                    help='aggregation top-down. E.g., GCN, SAGE, GAT, GIN, ...')
parser.add_argument('--fshot', dest='fshot', default=False, type=bool)

parser.set_defaults(gpu=False, task='LP', model='GCN', dataset='grid', feature_pre=True)
args = parser.parse_args()

In [3]:
args.task = 'LP'
args.dataset = 'grid'
args.mode = 'basemodel'
args.model = 'HCGNN'
args.layer_num = 3
args.epoch_num = 501
args.lr = 0.0001
args.relu = True
args.dropout = True
args.drop_ratio = 0.5
args.same_level_gnn = 'GIN'
args.down2up_gnn = 'MEAN'
args.up2down_gnn = 'GAT'
args.fshot = False
args.SEED = 123
args.gpu = True
args.device = torch.device('cuda:'+str(0) if args.gpu and torch.cuda.is_available() else 'cpu')

seed_everything(args.SEED)

print(args, '\n')

Namespace(SEED=123, community_detection_method='Louvain', dataset='grid', device=device(type='cuda', index=0), down2up_gnn='MEAN', drop_ratio=0.5, dropout=True, epoch_log=10, epoch_num=501, feature_pre=True, fshot=False, gpu=True, layer_num=3, lr=0.0001, mode='basemodel', model='HCGNN', ratio_sample_pos_link=20, relu=True, same_level_gnn='GIN', task='LP', threshold=1, up2down_gnn='GAT', use_features=True) 



In [4]:
ls_df_friends, graphs_complete, graphs, ls_valid_edges, ls_test_edges, features, df_labels = get_dataset(
    dataset_name=args.dataset,
    use_features=args.use_features,
    task=args.task,
    ratio_sample=args.ratio_sample_pos_link
)
ls_hierarchical_community, ls_up2down_edges, ls_down2up_edges = hierarchical_structure_generation(
    dataset_name=args.dataset,
    graphs=graphs,
    method=args.community_detection_method,
    threshold=args.threshold
)
ls_adj_same_level, ls_df_train, ls_df_valid, ls_df_test = LP_preparation(
    graphs=graphs,
    ls_df_friends=ls_df_friends,
    ls_test_edges=ls_test_edges,
    ls_valid_edges=ls_valid_edges,
    ls_hierarchical_community=ls_hierarchical_community
)
execute_LP(
    args, graphs, features, ls_hierarchical_community,
    ls_adj_same_level, ls_up2down_edges, ls_down2up_edges,
    ls_df_train, ls_df_valid, ls_df_test
)

is reading grid dataset...
datatset reading is done.
Name: 
Type: Graph
Number of nodes: 400
Number of edges: 760
Average degree:   3.8000
is processing dataset...
For graph 0, we need to collect 152 negative edges.
Generating 0 negative instances uses 0.10 seconds.
For graph 0, we need to remove 152 edges.
Generating 0 positive instances uses 0.10 seconds.
data processing is done
is generating hierarchical structure....
Is doing community detection....
start 0 subgraph...
iteration 0: from 400 items to 185 items, modularity is 0.35440426982340634.
iteration 1: from 185 items to 58 items, modularity is 0.6974117036011082.
iteration 2: from 58 items to 18 items, modularity is 0.8094499329120499.
iteration 3: from 18 items to 16 items, modularity is 0.8137065010387812.
Community detection is done
is presenting hierarchical structure....
start graph community 0
layer 0: keys: [400, 584], values: [0, 399]
layer 1: keys: [585, 642], values: [400, 584]
layer 2: keys: [643, 660], values: [585

Best valid performance is 0.8551, best test performance is 0.9037 and epoch_id is 190
Epoch 210 starts !
--------------------------------------------------------------------------------
Evaluating Epoch 210, time 0.084, ROC-AUC: Train = 0.9761, Valid = 0.8565, Test = 0.9055
Best valid performance is 0.8565, best test performance is 0.9055 and epoch_id is 210
Epoch 220 starts !
--------------------------------------------------------------------------------
Evaluating Epoch 220, time 0.082, ROC-AUC: Train = 0.9776, Valid = 0.8553, Test = 0.9034
Best valid performance is 0.8565, best test performance is 0.9055 and epoch_id is 210
Epoch 230 starts !
--------------------------------------------------------------------------------
Evaluating Epoch 230, time 0.084, ROC-AUC: Train = 0.9760, Valid = 0.8592, Test = 0.9067
Best valid performance is 0.8592, best test performance is 0.9067 and epoch_id is 230
Epoch 240 starts !
----------------------------------------------------------------------

Evaluating Epoch 500, time 0.127, ROC-AUC: Train = 0.9768, Valid = 0.8497, Test = 0.8852
Best valid performance is 0.8658, best test performance is 0.9065 and epoch_id is 270
